In [18]:
# Load prepared data
from src.graph.graph_generator import GraphGenerator

graph_generator = GraphGenerator()

roads_graph = graph_generator.read_graph("data/roads_w_attractions.pickle")
bike_paths_graph = graph_generator.read_graph("data/bike_paths_w_attractions.pickle")
attractions_graph = graph_generator.read_graph("data/attractions.pickle")

In [19]:
# Choose quality cycle paths
from src.graph.bike_paths import FULL_BIKE_PATH

whitelist = FULL_BIKE_PATH

In [20]:
print('\n'.join('- {}'.format(j) for i, j in enumerate(whitelist)))

- pas ruchu dla rowerów
- strefa ruchu uspokojonego 20 i 30 km/h
- kontraruch
- kontrapas
- droga dla rowerów
- trasa na wałach
- trasa przez park
- pas BUS + ROWER
- droga dla pieszych i rowerów


In [14]:
# Find edges which are not on the whitelist
edges_to_remove = []
for edge, attrib in bike_paths_graph.edges.items():
    if not (attrib.get("type") in whitelist or attrib.get("type") is None):
        edges_to_remove.append((edge[0], edge[1]))

# Remove previously selected edges. In case one of the nodes has no neighbors, delete it
for e1, e2 in edges_to_remove:
    bike_paths_graph.remove_edge(e1, e2)
    
    if not any(bike_paths_graph.neighbors(e1)):
        bike_paths_graph.remove_node(e1)
    if not any(bike_paths_graph.neighbors(e2)):
        bike_paths_graph.remove_node(e2)

In [21]:
from itertools import combinations
from src.utils import haversine

# Find different attraction pairs which are further than 1 km
attraction_pairs = []
for a1, a2 in combinations(attractions_graph.nodes.items(), 2):
    if haversine(*a1[1]["pos"], *a2[1]["pos"]) > 1.0:
        attraction_pairs.append((a1, a2))

In [22]:
from networkx.classes.function import path_weight
from IPython.display import clear_output
from src.algorithm.search_path import find_path

summary = []
for i, (a1, a2) in enumerate(attraction_pairs):
    bike_start = list(filter(lambda x: x[1].get("id") == a1[1]["id"], list(bike_paths_graph.nodes.items())))[0]
    bike_end = list(filter(lambda x: x[1].get("id") == a2[1]["id"], list(bike_paths_graph.nodes.items())))[0]

    try:
        bike_path = find_path(bike_paths_graph, bike_start[0], bike_end[0])
        bike_len = path_weight(bike_paths_graph, bike_path, weight="length")
    except:
        bike_len = float('inf')

    road_start = list(filter(lambda x: x[1].get("id") == a1[1]["id"], list(roads_graph.nodes.items())))[0]
    road_end = list(filter(lambda x: x[1].get("id") == a2[1]["id"], list(roads_graph.nodes.items())))[0]

    try:
        road_path = find_path(roads_graph, road_start[0], road_end[0])
        road_len = path_weight(roads_graph, road_path, weight="length")
    except:
        road_len = float('inf')
    
    summary.append(f"road: {'{:.2f}'.format(road_len)}\tbike: {'{:.2f}'.format(bike_len)}\tratio: {'{:.2f}'.format(road_len/bike_len)}")
    
    clear_output(wait=True)
    print(f"{int(i/len(attraction_pairs))}% ({i}/{len(attraction_pairs)})")

0% (20/109428)


KeyboardInterrupt: 

In [23]:
print('\n'.join('{}\t{}'.format(*i) for i in enumerate(summary)))

0	road: 2.84	bike: 3.57	ratio: 0.80
1	road: 3.26	bike: 4.06	ratio: 0.80
2	road: 3.15	bike: 4.17	ratio: 0.76
3	road: 3.43	bike: 4.38	ratio: 0.78
4	road: 3.38	bike: 4.29	ratio: 0.79
5	road: 3.36	bike: 4.35	ratio: 0.77
6	road: 3.18	bike: 4.01	ratio: 0.79
7	road: 3.45	bike: 4.16	ratio: 0.83
8	road: 4.87	bike: 5.84	ratio: 0.83
9	road: 3.55	bike: 4.49	ratio: 0.79
10	road: 3.19	bike: 4.10	ratio: 0.78
11	road: 3.26	bike: 4.10	ratio: 0.79
12	road: 3.97	bike: 4.75	ratio: 0.83
13	road: 3.59	bike: 4.51	ratio: 0.80
14	road: 5.21	bike: 6.06	ratio: 0.86
15	road: 5.60	bike: 6.82	ratio: 0.82
16	road: 3.63	bike: 3.99	ratio: 0.91
17	road: 3.12	bike: 3.86	ratio: 0.81
18	road: 5.49	bike: 6.65	ratio: 0.83
19	road: 7.59	bike: 8.48	ratio: 0.89
20	road: 7.32	bike: 8.43	ratio: 0.87
